In [19]:
# read input file
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [20]:
# collect all unique characters that appear in text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(vocab_size)
print(''.join(chars))

65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [21]:
# tokenize the text - convert raw text of strings to some sequence of integers
# according to some vocabulary of possible elements

# eg. for char level language model, we'll simply convert individual characters into integers

stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}
encode = lambda s: [stoi[c] for c in s] # encoder: takes input string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decode: takes a list of integers, output a string

print(encode('hii there'))
print(decode(encode('hii there')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [22]:
# encode the entire dataset and store in torch.tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)

# separating the dataset into train and val
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [23]:
# we never train a transformer on the complete data_set - computationally infeasible
# train on chunks of data == block_size

block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]  # offset by 1 i.e. data[1] follows data[0], making a complete set of (block_size) examples (x,y)
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'When input is {context}, target is {target}')

When input is tensor([18]), target is 47
When input is tensor([18, 47]), target is 56
When input is tensor([18, 47, 56]), target is 57
When input is tensor([18, 47, 56, 57]), target is 58
When input is tensor([18, 47, 56, 57, 58]), target is 1
When input is tensor([18, 47, 56, 57, 58,  1]), target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]), target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), target is 58


In [24]:
torch.manual_seed(1337)
batch_size = 4  # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+ block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('---------')

# these are (batch_size * block_size) 8*4=32 independent examples fed to the transformer

# for b in range(batch_size):  # batch dim
#     print(b)
#     for t in range(block_size):  # time dimension
#         context = xb[b, :t+1]
#         target = yb[b,t]
#         print(f'When input is {context.tolist()} the target: {target}')

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---------


In [25]:
# Implementing NN -- simplest is the bigram language model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx)  # (B,T,C)

        if targets == None:
            loss = None
        else:
            #pytorch expects logits in B,C,T dimensions i.e. Chanel dim as 2nd dim
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in current context
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)
            # focus only on last time step. The last elements in Time dim bcoz those are the predictions of what comes next
            # in this model, we are feeding the entire sequence of elements that came before 
            # then only looked at the last piece
            logits = logits[:, -1, :] # returns (B,C)
            # apply softmax to get probabilties
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
            

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=3)[0].tolist())

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


'\nSKI'

In [26]:
# training the model
# create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [27]:
batch_size = 32
for steps in range(10000):
    # sample batch of data
    xb, yb = get_batch('train')

    # evaluate loss
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.3836452960968018


In [28]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=300)[0].tolist()))


c wo the;
Pancalolinghowhatharean:
QA:

Wwhass bowoond;
Fomere d shdeenotep.
CI y mbotot swefesealso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee anf,
TOFonk? me ain ckntoty dedo bo'll


Mathematical trip in self-attention
* average of all the previous tokens

In [43]:
# example
import torch
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [48]:
# we need x[b,t] = mean_{i<=t} x[b,i], xbow == x bag of words
# inefficient method
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] #(t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [45]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3)) # creates a lower triangular matrix
a = a / torch.sum(a, 1, keepdim=True) # average of the rows
b = torch.randint(0,10, (3,2)).float()
c = a@b
print('a=')
print(a)
print('b=')
print(b)
print('c=') 
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [53]:
# working on the example
# version 2
wei = torch.tril(torch.ones(T,T))
wei /= wei.sum(1, keepdim=True)
# batch matrix multiply to do weighted aggregation
xbow2 = wei @ x   # (T, T) @ (B, T, C) ---> Broadcasting B dim to wei to make (B,T,T) @ (B,T,C) -> (B,T,C) 
torch.allclose(xbow, xbow2)


True

In [58]:
# version 3: use softmax
# better bcoz: wei zero matrix can be thought of as affinities between tokens at the start
# masked_fill is telling it that affinity is -inf. Future can't communicate with a past token
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril ==0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [14]:
import torch
x = torch.tensor([[1,2,3],[4,5,6]]).float()
print(x)
print(x.shape)
x.view(3, 2)


tensor([[1., 2., 3.],
        [4., 5., 6.]])
torch.Size([2, 3])


tensor([[1., 2.],
        [3., 4.],
        [5., 6.]])

In [17]:
y = x.transpose(1, 0).contiguous()  # transpose to get (2,3) shape
print(y.shape)
y.view(2, 3)  # reshaping to (2,3) to match original tensor's total elements

torch.Size([3, 2])


tensor([[1., 4., 2.],
        [5., 3., 6.]])

In [25]:
x = torch.ones(2,2,3)
y = torch.randint(10,(2,2,3)).float()
print(y)
x @ y.transpose(-2, -1)


tensor([[[6., 8., 8.],
         [9., 7., 4.]],

        [[8., 0., 2.],
         [2., 1., 4.]]])


tensor([[[22., 20.],
         [22., 20.]],

        [[10.,  7.],
         [10.,  7.]]])

Self Attention for single individual head
* attention is a communication information
* we want wei to be fairly diffused and not too high, else the softmax will get too peaky. Every other node is getting information from just a single node
* that's why scaling is used at initialization

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see single head perform self attention
head_size = 16
key = nn.Linear(C, head_size, bias=False) # what information I have?
query = nn.Linear(C, head_size, bias=False) # what am I looking for?
value = nn.Linear(C, head_size, bias=False) # what information I will return?
k = key(x)   # (B,T,head_size)
q = query(x) # (B,T,head_size)

wei = q @ k.transpose(-2, -1) # (B,T,head_size) @ (B,head_size,T) -> (B,T,T)


tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril ==0, float('-inf'))
wei = F.softmax(wei, dim=1)

v = value(x) 
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [30]:
wei[0]

tensor([[0.0248, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0052, 0.0091, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0521, 0.0135, 0.2482, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3171, 0.0214, 0.1642, 0.1188, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0412, 0.0487, 0.1046, 0.0742, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1060, 0.5347, 0.2059, 0.1030, 0.7402, 0.0192, 0.0000, 0.0000],
        [0.4298, 0.3409, 0.1769, 0.2027, 0.0480, 0.8472, 0.2329, 0.0000],
        [0.0238, 0.0316, 0.1002, 0.5013, 0.0117, 0.1336, 0.7671, 1.0000]],
       grad_fn=<SelectBackward0>)